# Wide&Deep

Wide&Deep 是 Google 在 2016 年发表于 DLRS 上的论文中提出来的，结合了 Wide 模型的 `Memorization` 和 Deep 模型的 `Generalization`。一句话介绍：  
**W&D由浅层（或单层）的Wide部分神经网络和深层的Deep部分多层神经网络组成，输出层采用softmax或logistics regression综合Wide和Deep部分的输出。**

![Wide & Deep](imgs/W&D.jpg)

如上图所示，W&D 由两部分组成：Wide 和 Deep。Wide 是一个广义的线性模型，用于捕捉在历史数据中出现过的模式 / 特征规律。Deep 是一个前馈神经网络，通过稀疏的类别特征学习特征稠密、低低维的 embedding，有更好的泛化性能。

Wide 是一个广义的线性模型，一般是逻辑回归模型：
$$
y_{wide} = \boldsymbol{w}^T \boldsymbol{x} + b,\ \boldsymbol{x} \in \mathbb{R}^d
$$
并且 Wide 部分可以加入人工构造的特征，即人工设计交叉特征，如交叉特征 $\phi_k$:
$$
\phi_K(\boldsymbol{x}) = \prod_{i=1}^d x_i^{c_{ki}}
$$
其中 $c_{ki}$ 表示第 $i$ 个特征 $x_i$ 是否出现在交叉特征 $\phi_k$ 中。

Deep 是一个前馈神经网络，一般是由多层神经网络组成。
$$
y_{deep} = DNN(\boldsymbol{x})
$$

最终二者相加后经过 `sigmoid` 激活函数得到最终的输出：
$$
P(Y = 1 | \boldsymbol{x}) = \hat{y} = \sigma(\boldsymbol{w}_{wide}^T [\boldsymbol{x}, \phi(\boldsymbol{x})] + \boldsymbol{w}^T_{deep} \boldsymbol{a}^{l_f} + b)
$$

其实从这个式子我们也可以看出，Wide&Deep 将输入 sigmoid 之前的回归值拆成了两部分，一部分来自 Wide 部分，一部分来自 Deep 部分。Wide 和 Deep 是联合训练的，通过梯度下降来更新参数，

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import itertools
from tqdm import tqdm
from collections import namedtuple

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  MinMaxScaler, LabelEncoder

from collections import namedtuple

In [6]:
data_path = "./data/criteo_sample.txt"

In [7]:
dataset = pd.read_csv(data_path)
dataset

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,NaN,3,260.0,NaN,17668.0,NaN,NaN,33.0,NaN,...,e5ba7672,87c6f83c,NaN,NaN,0429f84b,NaN,3a171ecb,c0d61a5c,NaN,NaN
1,0,NaN,-1,19.0,35.0,30251.0,247.0,1.0,35.0,160.0,...,d4bb7bd8,6fc84bfb,NaN,NaN,5155d8a3,NaN,be7c41b4,ded4aac9,NaN,NaN
2,0,0.0,0,2.0,12.0,2013.0,164.0,6.0,35.0,523.0,...,e5ba7672,675c9258,NaN,NaN,2e01979f,NaN,bcdee96c,6d5d1302,NaN,NaN
3,0,NaN,13,1.0,4.0,16836.0,200.0,5.0,4.0,29.0,...,e5ba7672,52e44668,NaN,NaN,e587c466,NaN,32c7478e,3b183c5c,NaN,NaN
4,0,0.0,0,104.0,27.0,1990.0,142.0,4.0,32.0,37.0,...,e5ba7672,25c88e42,21ddcdc9,b1252a9d,0e8585d2,NaN,32c7478e,0d4a6d1a,001f3601,92c878de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,NaN,0,113.0,3.0,3036.0,575.0,2.0,3.0,214.0,...,07c540c4,9880032b,21ddcdc9,5840adea,34cc61bb,c9d4222a,32c7478e,e5ed7da2,ea9a246c,984e0db0
196,1,0.0,1,1.0,1.0,1607.0,12.0,1.0,12.0,15.0,...,1e88c74f,3972b4ed,NaN,NaN,d1aa4512,NaN,32c7478e,9257f75f,NaN,NaN
197,1,1.0,0,6.0,3.0,0.0,0.0,19.0,3.0,3.0,...,3486227d,5aed7436,54591762,a458ea53,4a2c3526,NaN,32c7478e,1793a828,e8b83407,1a02cbe1
198,0,0.0,22,6.0,22.0,203.0,153.0,80.0,18.0,508.0,...,3486227d,13145934,55dd3565,5840adea,bf647035,NaN,32c7478e,1481ceb4,e8b83407,988b0775


# tensorflow 实现

参见：[WideNDeep-tf.py](./WideNDeep-tf.py)。该实现来自[这里](https://github.com/datawhalechina/fun-rec/blob/master/codes/base_models/WideNDeep.py)。

# pytorch 实现

参见：[Wide-and-Deep-PyTorch](https://github.com/zenwan/Wide-and-Deep-PyTorch)

In [1]:
from __future__ import print_function
import os
import torch
import numpy as np
import pandas as pd
from WideNDeep_pt import WideDeep
from data_utils import prepare_data


if __name__ == '__main__':

    use_cuda = torch.cuda.is_available()

    DF = pd.read_csv('data/adult.csv')
    DF['income_label'] = (DF["income"].apply(lambda x: ">50K" in x)).astype(int)

    # Experiment set up
    ## 定义 wide 部分使用的特征，一般是类别特征；定义交叉特征，一般是二元类别特征的交叉；定义 deep 部分需要 embedding 的特征，一般是类别特征；定义 deep 部分的连续值特征，一般是实数型特征。
    wide_cols = ['age','hours_per_week','education', 'relationship','workclass',
                 'occupation','native_country','gender']
    crossed_cols = (['education', 'occupation'], ['native_country', 'occupation'])
    embeddings_cols = [('education',10), ('relationship',8), ('workclass',10),
                        ('occupation',10),('native_country',10)]
    continuous_cols = ["age","hours_per_week"]
    target = 'income_label'
    method = 'logistic'

    # Prepare data
    wd_dataset = prepare_data(
        DF, wide_cols,
        crossed_cols,
        embeddings_cols,
        continuous_cols,
        target,
        scale=True)

    # Network set up
    wide_dim = wd_dataset['train_dataset'].wide.shape[1]
    n_unique = len(np.unique(wd_dataset['train_dataset'].labels))
    if (method=="regression") or (method=="logistic"):
        n_class = 1
    else:
        n_class = n_unique
    deep_column_idx = wd_dataset['deep_column_idx']  # dict, 类别特征 => 在 deep 数据中的索引
    embeddings_input= wd_dataset['embeddings_input']  # list, 每个元素为：(类别特征名字, 类别特征取值数, 类别特征表征长度)
    encoding_dict   = wd_dataset['encoding_dict']  # dict, 类别特征 => {取值 => idx}
    hidden_layers = [256, 128, 64]
    dropout = [0.5, 0.2, .2]

    model = WideDeep(
        wide_dim,
        embeddings_input,
        continuous_cols,
        deep_column_idx,
        hidden_layers,
        dropout,
        encoding_dict,
        n_class)
    model.compile(method=method)
    if use_cuda:
        model = model.cuda()

    train_dataset = wd_dataset['train_dataset']
    model.fit(dataset=train_dataset, n_epochs=30, batch_size=128)

    test_dataset  = wd_dataset['test_dataset']
    print((model.predict(dataset=test_dataset)==test_dataset.labels).sum() / len(test_dataset.labels))
    print(model.predict_proba(dataset=test_dataset)[:10])
    print(model.get_embeddings('education'))

    # save
#     MODEL_DIR = 'model'
#     if not os.path.exists(MODEL_DIR):
#         os.makedirs(MODEL_DIR)
#     torch.save(model.state_dict(), os.path.join(MODEL_DIR,'logistic.pkl'))

    # load model
    # model = WideDeep(
    #     wide_dim,
    #     embeddings_input,
    #     continuous_cols,
    #     deep_column_idx,
    #     hidden_layers,
    #     dropout,
    #     encoding_dict,
    #     n_class)
    # model.compile(method=method)
    # model.load_state_dict(torch.load('model/logistic.pkl'))

Epoch 1 of 30, Loss: 0.124, accuracy: 0.8114, auc: 0.8254855874544342
Epoch 2 of 30, Loss: 0.096, accuracy: 0.8355, auc: 0.8801040007735794
Epoch 3 of 30, Loss: 0.257, accuracy: 0.8386, auc: 0.8864987525289237
Epoch 4 of 30, Loss: 0.318, accuracy: 0.839, auc: 0.8896156035948188
Epoch 5 of 30, Loss: 0.531, accuracy: 0.8406, auc: 0.8920132984630393
Epoch 6 of 30, Loss: 0.348, accuracy: 0.844, auc: 0.8930158009125934
Epoch 7 of 30, Loss: 0.445, accuracy: 0.8466, auc: 0.8943107536601095
Epoch 8 of 30, Loss: 0.249, accuracy: 0.8442, auc: 0.8956991576959826
Epoch 9 of 30, Loss: 0.338, accuracy: 0.8458, auc: 0.8973800437512327
Epoch 10 of 30, Loss: 0.41, accuracy: 0.8467, auc: 0.8987414740106183
Epoch 11 of 30, Loss: 0.603, accuracy: 0.8478, auc: 0.8983957581701015
Epoch 12 of 30, Loss: 0.137, accuracy: 0.8484, auc: 0.8991320650794434
Epoch 13 of 30, Loss: 0.274, accuracy: 0.8492, auc: 0.8999786779222705
Epoch 14 of 30, Loss: 0.054, accuracy: 0.8472, auc: 0.9006401170054964
Epoch 15 of 30, Lo

# 总结
**一些问题：**
1. 为什么 Wide 侧会有记忆性？  
因为 Wide 侧的输入来源于类别特征，通常会对类别特征进行 one-hot 展开，而 Wide 侧是一个线性模型，会对不同特征有不同的权重，权重的大小反映了特征的强弱。在历史数据中出现过的一些特征组合会体现在权重的大小上，当再次出现类似的特征时，强的特征（组合）会获得更高的预测值，这就是 Wide 侧的记忆能力。Wide 部分学习的是历史数据中特征的共现频率，**如果一个特征组合于目标变量有很强的正相关，则该特征组合会获得较高的权重**。

2. 对于输入给 Wide 侧的数值特征，还会有记忆能力吗？  
xxx

3. 为什么推荐算法的网络层数都不高?   
网络层数太深会导致模型难以训练，误差反向传播过程中，越底层的网络训练难度越大。庞大的embedding层可能更加难以训练(好像有看到说对于embedding可能会采用不同的学习率，对于频繁更新的，学习率较小，而更新不频繁的，学习率较大）。推荐系统的网络并不是做不深，而是不能太深。原因有二，1) 深度的网络需要大量的信息来进行拟合的，而推荐系统的输入往往是一些高维度的稀疏特征，携带的信息有限，因此只需要较浅的网络就足够学习了，如果强行增加深度，反而网络不能收敛；2) 推荐系统往往是一个实时打分的系统，个人觉得，还是在保证效果的前提下，网络越简单越好，这样能保证请求延迟等各方面都处在比较好的程度。如果为了提高0.1%的效果，消耗了远远多与收益的成本，对于企业来说是不划算的。比如，xdeepfm，nffm，完全是堆积木。

4. Deep 侧的泛化能力是什么？  
Deep 的泛化能力指的是对于历史数据中未出现的或者很少的特征组合也能给出正确的预测，即使**出现了新的特征组合也能够正确推断出其对目标变量的贡献**。为什么 Deep 部分能泛化呢？Deep 侧对类别特征进行了表征，利用历史数据发掘特征之间隐藏的关系。deep部分的作用就是泛化，为的就是解决预测数据中出现wide部分没有包含的特征，尤其是ID组合特征。

5. 为什么Wide部分要用L1 FTRL训练？  
几年前FTRL曾风靡全部互联网头部公司，成为线性模型在线训练的主要方法。
简要来说，可以把FTRL当作一个**稀疏性**很好，精度又不错的随机梯度下降方法。由于是随机梯度下降，当然可以做到来一个样本就训练一次，进而实现模型的在线更新。所以在四五年前，大部分公司还是线性模型为主的时代，FTRL 凭借非常好的在线学习能力成为主流。问在这里“稀疏”这个性质又冒出来了。也就是说FTRL with L1非常注重模型的稀疏性。这也就是问题的答案，W&D 采用 L1 FTRL 是想让 Wide 部分变得更加稀疏。
再白话一点就是，L1 FTRL 会让 Wide 部分的大部分权重都为 0，我们**准备特征的时候就不用准备那么多0权重的特征了**，这大大压缩了模型权重，也压缩了特征向量的维度。

6. 为什么Deep部分不特别考虑稀疏性的问题？  
大家注意观察可以发现Deep部分的输入，要么是Age，#App Installs这些数值类特征，要么是已经降维并稠密化的Embedding向量，工程师们不会也不敢把过度稀疏的特征向量直接输入到Deep网络中。所以Deep部分不存在严重的特征稀疏问题，自然可以使用精度更好，更适用于深度学习训练的AdaGrad去训练。

7. 为什么L1正则化比L2正则化更容易产生稀疏解？
![从优化视角](./imgs/l1-l2_1.jpg)
![从梯度视角](./imgs/l1-l2_2.jpg)
![从概率视角](./imgs/l1-l2_3.jpg)

